In [ ]:
import pandas as pd
import glob
import os

from IPython.display import HTML, display
from cellphonedb.utils import db_releases_utils

In [ ]:
# 从adata准备所需数据（分组）
import scanpy as sc
import numpy as np
from scipy import stats
import os
from statsmodels.stats.multitest import multipletests

In [ ]:
display(HTML(db_releases_utils.get_remote_database_versions_html()['db_releases_html_table']))
os.chdir('/home/wyc')
cpdb_version = 'v5.0.0'

# -- Path where the input files to generate the database are located
cpdb_target_dir = os.path.join('./cellphonedb_v500_NatProtocol/', cpdb_version)
cpdb_target_dir
os.makedirs(cpdb_target_dir, exist_ok=True)
from cellphonedb.utils import db_utils
db_utils.download_database(cpdb_target_dir, cpdb_version)

In [ ]:
!pip list | grep scanpy
!pip list | grep cellphonedb

In [ ]:
adata= sc.read_h5ad('/home/wyc/AMD/01_cleandata/Final_Annotation.h5ad')

In [ ]:
os.chdir('/home/wyc/AMD/res/cellphoneDB')
output_dir = "cellphonedb_input"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# 创建输出目录
output_dir = "cellphonedb_input"
os.makedirs(output_dir, exist_ok=True)

# 获取分组信息
groups = adata.obs['age_disease'].dropna().unique()

# 为每个分组生成文件
for group in groups:
    print(f"\n===== 处理分组: {group} =====")
    group_dir = os.path.join(output_dir, group)
    os.makedirs(group_dir, exist_ok=True)
    
    # 1. 创建子集
    group_mask = adata.obs['age_disease'] == group
    adata_group = adata[group_mask].copy()
    
    # 2. 过滤细胞数量不足的细胞类型
    # 计算每个细胞类型的细胞数量
    celltype_counts = adata_group.obs['final_annotation'].value_counts()
    # 保留细胞数量≥20的细胞类型
    valid_celltypes = celltype_counts[celltype_counts >= 20].index.tolist()
    
    if not valid_celltypes:
        print(f"警告: 分组 {group} 中没有细胞类型满足≥20个细胞的条件，跳过此分组")
        continue
    
    # 应用过滤
    valid_mask = adata_group.obs['final_annotation'].isin(valid_celltypes)
    adata_group = adata_group[valid_mask].copy()
    
    print(f"保留细胞类型: {', '.join(valid_celltypes)}")
    print(f"细胞数量: {adata_group.n_obs}")
    
    # 3. 准备meta文件 (Cell, cell_type) - TSV格式
    meta_df = pd.DataFrame({
        'Cell': adata_group.obs_names,
        'cell_type': adata_group.obs['final_annotation']
    })
    meta_path = os.path.join(group_dir, f"{group}_meta.tsv")
    meta_df.to_csv(meta_path, sep='\t', index=False)
    print(f"已保存metadata文件: {meta_path}")
    
    # 4. 准备h5ad文件而不是counts文件
    # 保存当前子集为h5ad文件
    h5ad_path = os.path.join(group_dir, f"{group}.h5ad")
    adata_group.write(h5ad_path)
    print(f"已保存h5ad文件: {h5ad_path}")


print("\n===== 所有文件准备完成! =====")
print(f"输出目录结构:\n{output_dir}/")
for group in groups:
    group_dir = os.path.join(output_dir, group)
    if os.path.exists(group_dir):
        print(f"├── {group}/")
        print(f"│   ├── {group}.h5ad")
        print(f"│   └── {group}_meta.tsv")

In [ ]:
# 创建输出目录
output_dir = "cellphonedb_results"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

#分每组进行stastistical分析
for group in groups:
    print(f"\n===== 处理分组: {group} =====")
    group_dir = os.path.join(output_dir, group)
    os.makedirs(group_dir, exist_ok=True)
    
    # 获取输入文件路径
    h5ad_file_path = os.path.join('cellphonedb_input', group, f'{group}.h5ad')
    meta_file_path = os.path.join('cellphonedb_input', group, f'{group}_meta.tsv')
    
    # 使用Python API进行CellPhoneDB分析
    
    print(f"开始对分组 {group} 进行CellPhoneDB分析...")
    
    # 获取CellPhoneDB数据库文件路径
    cpdb_file_path = '/home/wyc/cellphonedb_v500_NatProtocol/v5.0.0/cellphonedb.zip'
    
    # 运行CellPhoneDB分析
    cpdb_results = cpdb_statistical_analysis_method.call(
        cpdb_file_path=cpdb_file_path,           # CellphoneDB数据库文件
        meta_file_path=meta_file_path,           # 元数据文件
        counts_file_path=h5ad_file_path,         # AnnData对象
        counts_data='hgnc_symbol',               # 基因注释类型
        score_interactions=True,                 # 对交互进行评分
        output_path=group_dir,                   # 结果保存路径
        separator='|',                           # 结果中分隔细胞的字符串
        threads=4,                               # 使用的线程数
        threshold=0.1,                           # 基因表达的最小细胞百分比阈值
        result_precision=3,                      # 结果精度
        debug=False,                             # 是否保存中间表
        output_suffix=f"{group}_results"         # 输出文件名后缀
    )
    
    print(f"已完成分组 {group} 的CellPhoneDB分析，结果保存在: {group_dir}")

In [ ]:
list(cpdb_results.keys())

In [ ]:
cpdb_results['significant_means'].head(2)  # pvalues means significant_means CellSign_active_interactions deconvoluted